In [27]:
%load_ext autoreload
%autoreload 2

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TextClassificationPipeline
from transformers import DataCollatorWithPadding

import torch
from pathlib import Path

classify_path = Path('/mnt/data/factcheck/tweets/classify')
tokenizer_path = Path(classify_path, 'models', 'xlm-roberta-tokenizer')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
# tokenizer.save_pretrained(tokenizer_path)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [28]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [29]:
from pathlib import Path
classify_path = Path('/mnt/data/factcheck/tweets/classify')

# CHANGE _model IF YOU WISH TO USE A DIFFERENT ONE FOR FILTERING
model_path = Path(classify_path, 'models', '_model')

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()
print('loaded')

cuda
loaded


In [30]:
import json
from pathlib import Path
from datasets import Dataset
import random

home_in_path = Path('/mnt/data/factcheck/twitter')
source_path = Path(home_in_path, 'people/text/tweets_with_titles.jsonl')
classify_path = Path('/mnt/data/factcheck/tweets/classify')

def load_jsonl(path):
    with open(path) as f:
        return [json.loads(line) for line in f]
    
source = load_jsonl(source_path)
print(source[0])

id_text_map = {item['id']: item['text'] for item in source}

def get_dataset(data):
    reordered = dict(
        text=[item['text'] for item in data],
        identifier=[item['id'] for item in data]
    )
    return Dataset.from_dict(reordered)

def tokenize_and_format_dataset(dataset):
    tokenize_fun = lambda example: tokenizer(example['text'], truncation=True)
    dataset = dataset.map(tokenize_fun, batched=True)
    dataset = dataset.remove_columns(['text'])
    dataset.set_format('torch')
    return dataset

data = get_dataset(source)
tokenized = tokenize_and_format_dataset(data)

{'id': 1422203548138450949, 'text': 'Dnes je poslední den pro podání žádosti o nový kompenzační bonus. Dosud vyplatila živnostníkům a dohodářům pomoc za více než 14 miliard korun 👍'}


  0%|          | 0/186 [00:00<?, ?ba/s]

In [31]:
from torch.utils.data import DataLoader
from itertools import compress
from tqdm.auto import tqdm
import json

dataloader = DataLoader(tokenized, batch_size=256, collate_fn=data_collator)

def evaluate(model, device, dataloader):
    result = []
    model.eval()
    progress_bar = tqdm(range(len(dataloader)))
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = outputs.logits.argmax(dim=-1)
        assert len(batch['identifier'])==len(predictions)
        result.extend([{'id':id_.item(),'text':id_text_map[id_.item()]} for 
                      id_, pred in zip(batch['identifier'], predictions) if pred])
        progress_bar.update(1)
    return result
        
res = evaluate(model, device, dataloader)
print(len(res))
path_plain = Path(classify_path, 'filtered.txt')
path_jsonl = Path(classify_path, 'filtered.jsonl')
with open(path_plain, 'wt') as fplain, open(path_jsonl, 'wt') as fjsonl:
    for r in res:
        print(r['text'],'\n', file=fplain)
        print(json.dumps(r, ensure_ascii=False), file=fjsonl)

  0%|          | 0/724 [00:00<?, ?it/s]

12808
